In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是由[Fabian Hirschmann](https://github.com/fhirschmann)贡献的一个更新版本。

# 在GCP上的E2E ML：MLOps阶段2：实验：使用R内核开始使用Vertex AI Training

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                        
</table>

## 概述

本示例演示了如何使用 R 内核（例如在 `Vertex AI Workbench Notebooks` 中）在 `Vertex AI` 上训练和部署 R 模型。

### 目标

在本教程中，您将学习如何使用`Vertex AI`，使用R内核来训练和部署一个自定义的R模型。

本教程使用以下谷歌云ML服务：

- `Vertex AI训练`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 创建一个自定义的R训练脚本
- 创建一个自定义的R服务脚本
- 创建一个自定义的R部署（服务）容器。
- 使用`Vertex AI`进行自定义训练来训练模型。
- 创建一个`端点`资源。
- 将`模型`资源（训练好的R模型）部署到`端点`资源。
- 进行在线预测。

### 数据集

本教程使用的数据集是[加利福尼亚房屋数据集](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html)。数据包含来自1990年加利福尼亚人口普查的信息。该数据集可以从Cloud Storage中公开获取，网址为`gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv`。该数据集用于训练随机森林回归器，以预测给定经度和纬度以及来自相应人口普查区块组的数据的房屋中位价格。区块组是美国人口普查局发布样本数据的最小地理单位（一个区块组通常有600到3000人口）。

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预估使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。
您需要以下内容:

* Google Cloud SDK
* Git
* R
* Python 3
* virtualenv
* 在带有R和Python 3的虚拟环境中运行的Jupyter笔记本

Cloud Storage指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简明的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装R]()。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用R的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

#### 为这个笔记本定义辅助函数

首先，您为本教程中使用的一些辅助函数定义如下：

- `sh`: 执行指定的命令shell。
- `display_file`: 显示指定文件的内容。

In [ ]:
library(glue)
library(IRdisplay)

sh <- function(cmd, args = c(), intern = FALSE) {
    if (is.null(args)) {
        cmd <- glue(cmd)
        s <- strsplit(cmd, " ")[[1]]
        cmd <- s[1]
        args <- s[2:length(s)]
    }
    ret <- system2(cmd, args, stdout = TRUE, stderr = TRUE)
    if ("errmsg" %in% attributes(attributes(ret))$names) cat(attr(ret, "errmsg"), "\n")
    if (intern) return(ret) else cat(paste(ret, collapse = "\n"))
}

display_file <- function(filename) {
    body <- sh(glue("pygmentize -g {filename} -f html -P full -O style=default"), intern = TRUE)
    IRdisplay::display_html(paste("<div style='text-align: left'>", paste(body, collapse="\n"), "</div>", collapse="\n"))
}

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
required_packages < -c("reticulate", "glue", "httr")
install.packages(setdiff(required_packages, rownames(installed.packages())))

sh("pip3 install --upgrade google-cloud-aiplatform -q")

在你开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要按照以下步骤进行操作。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。注册时，您将获得$300的免费信用额度，可用于计算/存储成本。

1. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)和[Artifact Registry API](https://console.cloud.google.com/flows/enableapi?apiid=artifactregistry.googleapis.com)。

1. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 自动检测项目ID或在下方输入。然后运行单元格，确保Cloud SDK在这个笔记本中的所有命令中使用正确的项目。

设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID < -"[your-project-id]"  # @param {type:"string"}

In [ ]:
# Get your Google Cloud project ID from gcloud
if (Sys.getenv("IS_TESTING") == "") {
    if (PROJECT_ID == "[your-project-id]") {
      PROJECT_ID <- sh("gcloud config list --format 'value(core.project)' 2>/dev/null", intern = TRUE)
    }
    cat("Project ID:", PROJECT_ID, "\n")
}

In [ ]:
sh("gcloud config set project {PROJECT_ID}")

#### 区域

您也可以更改`REGION`变量，该变量用于本笔记本中的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能在Vertex AI上使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION <- "[your-region]"  # @param {type: "string"}

if (REGION == "[your-region]") {
    REGION <- "us-central1"
}

时间戳

如果你在一个实时教程会话中，你可能使用一个共享的测试账户或项目。为了避免在创建的资源上出现命名冲突，你为每个实例会话创建一个时间戳，并将它附加到你在本教程中创建的资源的名称上。

In [ ]:
TIMESTAMP = format(Sys.time(), "%Y%m%d%H%M%S")

###验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行账户验证。

**否则**，按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务账号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账号**。

3. 在**服务账号名称**字段中输入名称，并点击**创建**。

4. 在**授予此服务账号访问项目的权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，选择**存储对象管理员**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中将您的服务账号密钥路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行该单元格。

In [ ]:
# The Google Cloud Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK <- file.exists("/opt/deeplearning/metadata/env_version")
IS_GOOGLE_COLAB <- (system("python -c 'import google.colab'") != 0)

GOOGLE_APPLICATION_CREDENTIALS <- ""  # @param {type:"string"}

# If on Vertex AI Workbench Notebooks, then don't execute this code
if (!IS_WORKBENCH_NOTEBOOK) {
    # Replace the string below with the path to your service account key and run this cell
    # to authenticate your GCP account.
    Sys.setenv("GOOGLE_APPLICATION_CREDENTIALS" = GOOGLE_APPLICATION_CREDENTIALS)
}

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要按照以下步骤操作。**

当您在Python中初始化Vertex AI SDK时，您需要指定一个云存储暂存桶。暂存桶是在会话之间保留与您的数据集和模型资源相关的所有数据的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME < -"[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI < -paste0("gs://", BUCKET_NAME)

In [ ]:
if (BUCKET_NAME == "" || BUCKET_NAME == "[your-bucket-name]") {
    BUCKET_NAME = paste0(PROJECT_ID, "aip-", TIMESTAMP)
    BUCKET_URI = paste0("gs://", BUCKET_NAME)
}

BUCKET_URI

只要您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
sh("gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}")

最后，通过检查Cloud Storage存储桶的内容来验证对其的访问权限。

In [ ]:
sh("gsutil ls -al {BUCKET_URI}")

### 导入库并定义常量

导入并初始化`reticulate` R包，用于与使用Python编写的Vertex AI SDK进行交互。

In [ ]:
library(reticulate)
library(glue)
library(httr)
library(IRdisplay)

use_python(Sys.which("python3"))

aiplatform <- import("google.cloud.aiplatform")

### 为 Python 初始化 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform$init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_URI)

## Google Artifact Registry简介

`Google Artifact Registry`是一个用于存储和管理私有存储库中的artifact（软件构件）的服务，包括容器镜像、Helm图表和语言包。它是Google Cloud推荐的容器镜像注册表。

了解更多关于[Docker的快速入门](https://cloud.google.com/artifact-registry/docs/docker/quickstart)。

### 启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于 [启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
sh("gcloud services enable artifactregistry.googleapis.com")

## 创建一个私有的Docker存储库

首先，您需要在Google Artifact Registry中创建自己的Docker存储库。

1. 运行`gcloud artifacts repositories create`命令，在您所在地区创建一个新的Docker存储库，其中包含描述为"docker repository"。

2. 运行`gcloud artifacts repositories list`命令来验证您的存储库是否已创建。

In [ ]:
PRIVATE_REPO < -"my-docker-repo"

sh(
    'gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"'
)

sh("gcloud artifacts repositories list")

### 配置私有存储库的身份验证

在推送或拉取容器映像之前，配置Docker以使用`gcloud`命令行工具对您所在地区的`Artifact Registry`进行身份验证请求。

In [ ]:
sh("gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet")

### 创建 Dockerfile

您定制容器的 Docker 文件是构建在深度学习容器的基础上的 - 这也是 Vertex AI Workbench 所使用的容器。另外，您还添加了两个 R 脚本分别用于模型训练和服务。

In [ ]:
IMAGE_NAME < -"vertex-r"  # @param {type:"string"}
IMAGE_TAG < -"latest"  # @param {type:"string"}
IMAGE_URI < -glue(
    "{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
)

dir.create("src", showWarnings=FALSE)

In [ ]:
Dockerfile = cat("
# filename: Dockerfile - container specifications for using R in Vertex AI
FROM gcr.io/deeplearning-platform-release/r-cpu.4-1:latest

WORKDIR /root

COPY train.R /root/train.R
COPY serve.R /root/serve.R

# Install Fortran
RUN apt-get update
RUN apt-get install gfortran -yy

# Install R packages
RUN Rscript -e \"install.packages('plumber')\"
RUN Rscript -e \"install.packages('randomForest')\"

EXPOSE 8080
", file = "src/Dockerfile")

display_file("src/Dockerfile")

### 创建训练脚本

接下来，创建名为 `train.R` 的文件，用于训练您的 R 模型。该脚本在加利福尼亚房屋数据集上训练一个 `randomForest` 模型。Vertex AI 设置环境变量供您使用，并且由于此脚本使用了 Vertex AI 管理的数据集，数据拆分由 Vertex AI 执行，脚本接收指向训练、测试和验证集的环境变量。

然后将训练好的模型工件存储到您的 Cloud 存储桶中。

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: train.R - train a Random Forest model on Vertex AI Managed Dataset
library(tidyverse)
library(data.table)
library(randomForest)
Sys.getenv()

# The GCP Project ID
project_id <- Sys.getenv("CLOUD_ML_PROJECT_ID")

# The GCP Region
location <- Sys.getenv("CLOUD_ML_REGION")

# The Cloud Storage URI to upload the trained model artifact to
model_dir <- Sys.getenv("AIP_MODEL_DIR")

# Next, you create directories to download our training, validation, and test set into.
dir.create("training")
dir.create("validation")
dir.create("test")

# You download the Vertex AI managed data sets into the container environment locally.
system2("gsutil", c("cp", Sys.getenv("AIP_TRAINING_DATA_URI"), "training/"))
system2("gsutil", c("cp", Sys.getenv("AIP_VALIDATION_DATA_URI"), "validation/"))
system2("gsutil", c("cp", Sys.getenv("AIP_TEST_DATA_URI"), "test/"))

# For each data set, you may receive one or more CSV files that you will read into data frames.
training_df <- list.files("training", full.names = TRUE) %>% map_df(~fread(.))
validation_df <- list.files("validation", full.names = TRUE) %>% map_df(~fread(.))
test_df <- list.files("test", full.names = TRUE) %>% map_df(~fread(.))

print("Starting Model Training")
rf <- randomForest(median_house_value ~ ., data=training_df, ntree=100)
rf

saveRDS(rf, "rf.rds")
system2("gsutil", c("cp", "rf.rds", model_dir))
', file = "src/train.R")

display_file("src/train.R")

### 创建服务脚本

接下来，创建文件`serve.R`，用于为您的R模型提供服务。该脚本从云存储下载模型文件，加载模型文件，并在端口`8080`上监听预测请求。

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: serve.R - serve predictions from a Random Forest model
Sys.getenv()
library(plumber)

system2("gsutil", c("cp", "-r", Sys.getenv("AIP_STORAGE_URI"), "."))
system("du -a .")

rf <- readRDS("artifacts/rf.rds")
library(randomForest)

predict_route <- function(req, res) {
    print("Handling prediction request")
    df <- as.data.frame(req$body$instances)
    preds <- predict(rf, df)
    return(list(predictions=preds))
}

print("Staring Serving")

pr() %>%
    pr_get(Sys.getenv("AIP_HEALTH_ROUTE"), function() "OK") %>%
    pr_post(Sys.getenv("AIP_PREDICT_ROUTE"), predict_route) %>%
    pr_run(host = "0.0.0.0", port=as.integer(Sys.getenv("AIP_HTTP_PORT", 8080)))
', file = "src/serve.R")

display_file("src/serve.R")

构建Docker容器

接下来，在Cloud Build上构建Docker容器镜像 -- 这是无服务器的CI/CD平台。

*注意:* 构建Docker容器镜像可能需要10至15分钟。

In [ ]:
sh("cd src && gcloud builds submit --region={REGION} --tag={IMAGE_URI} --timeout=1h")

创建 Vertex AI 托管数据集

您可以创建 Vertex AI 托管数据集，让 Vertex AI 负责数据集拆分。这是可选的，作为替代，您也可以通过环境变量传递数据集的 URI。

In [ ]:
data_uri <- "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv"

dataset <- aiplatform$TabularDataset$create(
    display_name = "California Housing Dataset",
    gcs_source = data_uri
)

## 创建一个 Vertex AI 自定义训练作业

自定义训练作业通过创建您的容器映像的实例并执行 `train.R` 来包装训练过程进行模型训练，并执行 `serve.R` 进行模型服务。

*注意:* 您在训练和服务中使用相同的自定义容器。

In [ ]:
job <- aiplatform$CustomContainerTrainingJob(
    display_name = "vertex-r",
    container_uri = IMAGE_URI,
    command = c("Rscript", "train.R"),
    model_serving_container_command = c("Rscript", "serve.R"),
    model_serving_container_image_uri = IMAGE_URI
)

### 执行自定义训练作业

要训练模型，您需要调用`run()`方法，选择一个资源充足的机器类型来训练您的数据集上的机器学习模型。在本教程中，您可以使用`n1-standard-4`虚拟机实例。

In [ ]:
model <- job$run(
    dataset=dataset,
    model_display_name = "vertex-r-model",
    machine_type = "n1-standard-4"
)

model$display_name
model$resource_name
model$uri

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法来创建一个 `Endpoint` 资源。至少，您需要指定端点的显示名称。可以选择性地指定项目和位置（地区）；否则，设置将继承您在使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在这个示例中，以下参数被指定：

- `display_name`：`Endpoint` 资源的可读名称。
- `project`：您的项目 ID。
- `location`：您的地区。
- `labels`：（可选）用户定义的以键/值对形式的 `Endpoint` 元数据。

此方法返回一个 `Endpoint` 对象。

了解更多关于 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint <- aiplatform$Endpoint$create(
    display_name = "California Housing Endpoint",
    project = PROJECT_ID,
    location = REGION
)

endpoint

部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个终端。 每个部署的 `Vertex AI Model` 资源将拥有自己的用于服务二进制的部署容器。

接下来，您将 `Vertex AI Model` 资源部署到 `Vertex AI Endpoint` 资源。 `Vertex AI Model` 资源已经为它定义了部署容器图像。 要部署，您需要指定以下附加配置设置：

- 机器类型。
- GPU 的类型和数量（如果有的话）。
- VM 实例的静态、手动或自动扩展。

在此示例中，您将使用最少量的指定参数部署模型，如下所示：

- `model`：`Model` 资源。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于需要为资源进行配置，可能需要几分钟时间。

*注意：* 在此示例中，在将模型工件上传到 `Vertex AI Model` 资源的前一步中，您已指定了 R 部署容器。

In [ ]:
model$deploy(endpoint = endpoint, machine_type = "n1-standard-4")

### 测试部署的模型

现在您的 `Model`资源已经部署到一个`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 创建示例数据

首先，您需要创建一些示例数据，以便测试向部署的模型发出预测请求。您可以使用来自`data_uri`中原始数据文件的五个经过JSON编码的示例数据点（不含标签`median_house_value`）。

In [ ]:
df <- read.csv(text=sh("gsutil cat {data_uri}", intern = TRUE))
head(df, 5)

instances = list(instances=head(df[, names(df) != "median_house_value"], 5))
instances

json_instances = toJSON(instances)
json_instances

#### 发起预测请求

最后，您使用示例数据发起预测请求。在这个例子中，您可以使用 REST API（例如 Curl） 发起预测请求。

In [ ]:
url < -glue(
    "https://{REGION}-aiplatform.googleapis.com/v1/{endpoint$resource_name}:predict"
)
access_token < -sh("gcloud auth print-access-token", intern=TRUE)

sh(
    "curl",
    c(
        "--tr-encoding",
        "-s",
        "-X POST",
        glue("-H 'Authorization: Bearer {access_token}'"),
        "-H 'Content-Type: application/jsoin'",
        url,
        glue("-d {json_instances}"),
    ),
)

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = FALSE

endpoint$undeploy_all()
endpoint$delete()
dataset$delete()
model$delete()
job$delete()

sh("rm -rf src")

# remove Docker image
sh("docker rmi {IMAGE_URI}")

if (delete_bucket == TRUE) sh("gsutil -m rm -r {BUCKET_URI}")